In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from IPython.display import display, Image
import time
import ipywidgets as widgets
from tqdm.notebook import tqdm

# Load the Stable Diffusion model
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

def generate_image(prompt):
    start_time = time.time()
    
    with tqdm(total=100, desc="Generating image", bar_format='{l_bar}{bar}') as pbar:
        for i in range(10):
            time.sleep(0.1)  # Simulate steps in generation process
            pbar.update(10)
        image = pipe(prompt).images[0]
    
    end_time = time.time()
    
    # Save the image
    image_path = f"generated_image_{int(time.time())}.png"
    image.save(image_path)
    
    print(f"Image generated in {end_time - start_time:.2f} seconds")
    print(f"Image saved as {image_path}")
    
    # Display the image
    display(Image(filename=image_path))

# Create widgets for user interaction
text_input = widgets.Text(description="Image description:", style={'description_width': 'initial'})
generate_button = widgets.Button(description="Generate")
output = widgets.Output()

def on_button_click(b):
    with output:
        output.clear_output()
        generate_image(text_input.value)

generate_button.on_click(on_button_click)

# Display the UI
display(widgets.VBox([text_input, generate_button, output]))
